In [1]:
using ModelingToolkit
using MethodOfLines
using DifferentialEquations
using DomainSets
using FFTW
using Plots

In [2]:
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
t0::Float64 = 0.0;
T::Float64 = 10.0;
dt = 0.01;
Nx = 200;
Nt = T / dt;
order = 2;
step = (xright - xleft)/Nx;
grid = collect(xleft:step:xright);

g0::Float64 = 9.81; # m / s^2
height = 0.1; # m
omega = 2*pi*1
gamma = 0.0
gamma_3 = 0.0
A = 5.0


5.0

In [3]:
function f1!(dv, v, u, p, t)
    dx, c, γ, γ3, A, omega = p
    N = length(u)
    invdx2 = 1 / dx^2

    # interior points
    @inbounds for i in 2:N-1
        uxx = (u[i+1] - 2u[i] + u[i-1]) * invdx2
        dv[i] = c^2 * uxx - γ * v[i] - γ3 * v[i]^3 + A * sin(omega * t)
    end
    dv[1] = 0
    dv[end] = 0
    

end


function f2!(du, v, u, p, t)
    dx, c, γ, γ3, A, omega = p
    N = length(u)
    du .= v
    du[1] = 0
    du[end] = 0
end



f2! (generic function with 1 method)

In [4]:
u0 = zeros(Nx+1)
v0 = zeros(Nx+1)

tspan = (0.0, 10.0)
p = (step, sqrt(g0 * height), gamma, gamma_3, A, omega)


(0.005, 0.9904544411531507, 0.0, 0.0, 5.0, 6.283185307179586)

In [5]:
prob = DynamicalODEProblem(f1!, f2!, u0, v0, tspan, p)
solution = solve(prob, VelocityVerlet(), reltol = 1e-8, abstol = 1e-8, dt = 0.001);


In [6]:
U = hcat([sol.x[1] for sol in solution.u]...)
sol_u = U


201×10001 Matrix{Float64}:
 0.0  0.0         0.0         0.0          …  0.0        0.0        0.0
 0.0  1.57079e-5  6.22144e-5  0.000137717     0.0143891  0.0142688  0.0141484
 0.0  1.57079e-5  6.28308e-5  0.000141343     0.0279832  0.0277363  0.0274898
 0.0  1.57079e-5  6.28308e-5  0.000141367     0.0408008  0.0404107  0.0400206
 0.0  1.57079e-5  6.28308e-5  0.000141367     0.0528164  0.052284   0.0517501
 0.0  1.57079e-5  6.28308e-5  0.000141367  …  0.0639819  0.0633193  0.0626526
 0.0  1.57079e-5  6.28308e-5  0.000141367     0.074238   0.0734504  0.0726579
 0.0  1.57079e-5  6.28308e-5  0.000141367     0.0835602  0.0826582  0.0817537
 0.0  1.57079e-5  6.28308e-5  0.000141367     0.0920258  0.0910004  0.0899736
 0.0  1.57079e-5  6.28308e-5  0.000141367     0.0996605  0.0985085  0.0973555
 0.0  1.57079e-5  6.28308e-5  0.000141367  …  0.106457   0.105191   0.103922
 0.0  1.57079e-5  6.28308e-5  0.000141367     0.11241    0.111025   0.109637
 0.0  1.57079e-5  6.28308e-5  0.000141367    

In [ ]:
# Your solution data
discrete_t = solution.t
grid = grid
hcat([sol.x[1] for sol in solution.u]...)
Nx = length(grid)
Nt = length(discrete_t)

# Create the animation object
anim = @animate for k in 1:100:Nt
    t = discrete_t[k]
    plot(
        grid, solution.u[k].x[1],
        ylim = (-2, 2),
        xlabel = "x",
        ylabel = "u(x, t)",
        title = "t = $(round(t, digits=2))",
        legend = false
    )
end

# Save the animation as a GIF
gif(anim, "solution_animation_1.gif", fps = 10)